In [2]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [29]:
#df_train = pd.read_csv("data_source/Champions-League-Scores-and-Fixture.csv") #for the input 2016-2023
df_train = pd.read_csv("data_source/Champions-League-Scores-and-Fixture_2021_2022.csv") #for the input 2021-2023
df_test= pd.read_csv("data_source/Champions-League-to-predict.csv")

In [30]:
df_train.head()

,round,matchweek_no,date,time,home_team,expected_goals_home,score,expected_goals_away,away_team,attendance,stadium,referee,home_team_score,away_team_score,winner,season
0,Group stage,1,2022-09-06,18:45,Dinamo Zagreb,0.6,1–0,0.9,Chelsea,20607,Stadion Maksimir,István Kovács,1,0,Dinamo Zagreb,2022-2023
1,Group stage,1,2022-09-06,18:45,Dortmund,2.3,3–0,0.5,FC Copenhagen,70700,Signal Iduna Park,François Letexier,3,0,Dortmund,2022-2023
2,Group stage,1,2022-09-06,20:00,Celtic,1.4,0–3,1.9,Real Madrid,57057,Celtic Park,Sandro Schärer,0,3,Real Madrid,2022-2023
3,Group stage,1,2022-09-06,20:00,Benfica,1.1,2–0,0.2,Maccabi Haifa,55130,Estádio do Sport Lisboa e Benfica,Andreas Ekberg,2,0,Benfica,2022-2023
4,Group stage,1,2022-09-06,21:00,RB Salzburg,1.3,1–1,0.9,Milan,29520,Red Bull Arena,Srđan Jovanović,1,1,draw,2022-2023


In [31]:
#df_train['date'] = pd.to_datetime(df_train['date'])
#df_train = df_train[df_train['date'] > pd.to_datetime('2022-09-01')]

In [32]:
df_test

,round,matchweek_no,date,time,home_team,expected_goals_home,score,expected_goals_away,away_team,attendance,stadium,referee,home_team_score,away_team_score,winner,season
0,Quarter-finals,0,2023-04-11,20:00,Benfica,NaN,NaN,NaN,Inter,0,Estádio da Luz (1954),NaN,0,0,draw,2022-2023
1,Quarter-finals,0,2023-04-11,20:00,Manchester City,NaN,NaN,NaN,Bayern Munich,0,Etihad Stadium,NaN,0,0,draw,2022-2023
2,Quarter-finals,0,2023-04-12,21:00,Real Madrid,NaN,NaN,NaN,Chelsea,0,Estadio Santiago Bernabéu,NaN,0,0,draw,2022-2023
3,Quarter-finals,0,2023-04-12,21:00,Milan,NaN,NaN,NaN,Napoli,0,Stadio Giuseppe Meazza,NaN,0,0,draw,2022-2023
4,Quarter-finals,0,2023-04-18,20:00,Chelsea,NaN,NaN,NaN,Real Madrid,0,Stamford Bridge,NaN,0,0,draw,2022-2023
5,Quarter-finals,0,2023-04-18,21:00,Napoli,NaN,NaN,NaN,Milan,0,Stadio Diego Armando Maradona,NaN,0,0,draw,2022-2023
6,Quarter-finals,0,2023-04-19,21:00,Bayern Munich,NaN,NaN,NaN,Manchester City,0,Allianz Arena,NaN,0,0,draw,2022-2023
7,Quarter-finals,0,2023-04-19,21:00,Inter,NaN,NaN,NaN,Benfica,0,Stadio Giuseppe Meazza,NaN,0,0,draw,2022-2023


In [33]:
df_test.drop(columns=['expected_goals_home', 'score', 'expected_goals_away', 'attendance', 'referee', ], inplace = True)

# Calculate Team Strength

In [34]:
df_home = df_train[['home_team', 'home_team_score', 'away_team_score']]
df_away = df_train[['away_team', 'home_team_score', 'away_team_score']]

df_home = df_home.rename(columns={'home_team':'Team', 'home_team_score': 'GoalsScored', 'away_team_score': 'GoalsConceded'})
df_away = df_away.rename(columns={'away_team':'Team', 'home_team_score': 'GoalsConceded', 'away_team_score': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Ajax,1.833333,2.666667
Atlético Madrid,0.833333,1.500000
Barcelona,2.000000,2.000000
Bayern Munich,2.625000,0.250000
Benfica,2.875000,1.000000
Celtic,0.666667,2.500000
Chelsea,1.500000,0.625000
Club Brugge,1.000000,1.375000
Dinamo Zagreb,0.666667,1.833333


# Function predict_points

In [36]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

# Predicting

In [55]:
df_test

,round,matchweek_no,date,time,home_team,expected_goals_home,score,expected_goals_away,away_team,attendance,stadium,referee,home_team_score,away_team_score,winner,season
0,Quarter-finals,0,2023-04-11,20:00,Benfica,NaN,NaN,NaN,Inter,0,Estádio da Luz (1954),NaN,0,0,draw,2022-2023
1,Quarter-finals,0,2023-04-11,20:00,Manchester City,NaN,NaN,NaN,Bayern Munich,0,Etihad Stadium,NaN,0,0,draw,2022-2023
2,Quarter-finals,0,2023-04-12,21:00,Real Madrid,NaN,NaN,NaN,Chelsea,0,Estadio Santiago Bernabéu,NaN,0,0,draw,2022-2023
3,Quarter-finals,0,2023-04-12,21:00,Milan,NaN,NaN,NaN,Napoli,0,Stadio Giuseppe Meazza,NaN,0,0,draw,2022-2023
4,Quarter-finals,0,2023-04-18,20:00,Chelsea,NaN,NaN,NaN,Real Madrid,0,Stamford Bridge,NaN,0,0,draw,2022-2023
5,Quarter-finals,0,2023-04-18,21:00,Napoli,NaN,NaN,NaN,Milan,0,Stadio Diego Armando Maradona,NaN,0,0,draw,2022-2023
6,Quarter-finals,0,2023-04-19,21:00,Bayern Munich,NaN,NaN,NaN,Manchester City,0,Allianz Arena,NaN,0,0,draw,2022-2023
7,Quarter-finals,0,2023-04-19,21:00,Inter,NaN,NaN,NaN,Benfica,0,Stadio Giuseppe Meazza,NaN,0,0,draw,2022-2023


In [64]:
print(predict_points('Benfica', 'Maccabi Haifa')[0])
type(predict_points('Manchester City', 'Real Madrid'))

1.7192715339120355


tuple

In [75]:
for index, row in df_test.iterrows():
    home, away = row['home_team'], row['away_team']
    t = predict_points(home, away)
    index_of_larger_value = t.index(max(t))
    df_test.at[index, "home_team_score"] += t[0]
    df_test.at[index, "away_team_score"] += t[1]
    df_test.at[index, "winner"] = [home, away][index_of_larger_value]

In [76]:
#for data: 2016-2023
df_test

,round,matchweek_no,date,time,home_team,expected_goals_home,score,expected_goals_away,away_team,attendance,stadium,referee,home_team_score,away_team_score,winner,season
0,Quarter-finals,0,2023-04-11,20:00,Benfica,NaN,NaN,NaN,Inter,0,Estádio da Luz (1954),NaN,2.419257,0.451951,Benfica,2022-2023
1,Quarter-finals,0,2023-04-11,20:00,Manchester City,NaN,NaN,NaN,Bayern Munich,0,Etihad Stadium,NaN,0.280179,2.600663,Bayern Munich,2022-2023
2,Quarter-finals,0,2023-04-12,21:00,Real Madrid,NaN,NaN,NaN,Chelsea,0,Estadio Santiago Bernabéu,NaN,1.113693,1.659458,Chelsea,2022-2023
3,Quarter-finals,0,2023-04-12,21:00,Milan,NaN,NaN,NaN,Napoli,0,Stadio Giuseppe Meazza,NaN,0.599859,2.238840,Napoli,2022-2023
4,Quarter-finals,0,2023-04-18,20:00,Chelsea,NaN,NaN,NaN,Real Madrid,0,Stamford Bridge,NaN,1.659458,1.113693,Chelsea,2022-2023
5,Quarter-finals,0,2023-04-18,21:00,Napoli,NaN,NaN,NaN,Milan,0,Stadio Diego Armando Maradona,NaN,2.238840,0.599859,Napoli,2022-2023
6,Quarter-finals,0,2023-04-19,21:00,Bayern Munich,NaN,NaN,NaN,Manchester City,0,Allianz Arena,NaN,2.600663,0.280179,Bayern Munich,2022-2023
7,Quarter-finals,0,2023-04-19,21:00,Inter,NaN,NaN,NaN,Benfica,0,Stadio Giuseppe Meazza,NaN,0.451951,2.419257,Benfica,2022-2023


In [37]:
for index, row in df_test.iterrows():
    home, away = row['home_team'], row['away_team']
    t = predict_points(home, away)
    index_of_larger_value = t.index(max(t))
    df_test.at[index, "home_team_score"] += t[0]
    df_test.at[index, "away_team_score"] += t[1]
    df_test.at[index, "winner"] = [home, away][index_of_larger_value]

In [91]:
#for input data: 2021-2023
df_test

,round,matchweek_no,date,time,home_team,away_team,stadium,home_team_score,away_team_score,winner,season
0,Quarter-finals,0,2023-04-11,20:00,Benfica,Inter,Estádio da Luz (1954),4.099933,1.536540,Benfica,2022-2023
1,Quarter-finals,0,2023-04-11,20:00,Manchester City,Bayern Munich,Etihad Stadium,2.162925,3.160564,Bayern Munich,2022-2023
2,Quarter-finals,0,2023-04-12,21:00,Real Madrid,Chelsea,Estadio Santiago Bernabéu,2.945481,2.582209,Real Madrid,2022-2023
3,Quarter-finals,0,2023-04-12,21:00,Milan,Napoli,Stadio Giuseppe Meazza,1.199718,4.477681,Napoli,2022-2023
4,Quarter-finals,0,2023-04-18,20:00,Chelsea,Real Madrid,Stamford Bridge,2.582209,2.945481,Real Madrid,2022-2023
5,Quarter-finals,0,2023-04-18,21:00,Napoli,Milan,Stadio Diego Armando Maradona,4.477681,1.199718,Napoli,2022-2023
6,Quarter-finals,0,2023-04-19,21:00,Bayern Munich,Manchester City,Allianz Arena,3.160564,2.162925,Bayern Munich,2022-2023
7,Quarter-finals,0,2023-04-19,21:00,Inter,Benfica,Stadio Giuseppe Meazza,1.536540,4.099933,Benfica,2022-2023


In [24]:
#for input data: 2023
df_test

,round,matchweek_no,date,time,home_team,away_team,stadium,home_team_score,away_team_score,winner,season
0,Quarter-finals,0,2023-04-11,20:00,Benfica,Inter,Estádio da Luz (1954),0.778801,1.442398,Inter,2022-2023
1,Quarter-finals,0,2023-04-11,20:00,Manchester City,Bayern Munich,Etihad Stadium,0.472367,2.055267,Bayern Munich,2022-2023
2,Quarter-finals,0,2023-04-12,21:00,Real Madrid,Chelsea,Estadio Santiago Bernabéu,1.723683,1.016468,Real Madrid,2022-2023
3,Quarter-finals,0,2023-04-12,21:00,Milan,Napoli,Stadio Giuseppe Meazza,1.000000,1.000000,Milan,2022-2023
4,Quarter-finals,0,2023-04-18,20:00,Chelsea,Real Madrid,Stamford Bridge,1.016468,1.723683,Real Madrid,2022-2023
5,Quarter-finals,0,2023-04-18,21:00,Napoli,Milan,Stadio Diego Armando Maradona,1.000000,1.000000,Napoli,2022-2023
6,Quarter-finals,0,2023-04-19,21:00,Bayern Munich,Manchester City,Allianz Arena,2.055267,0.472367,Bayern Munich,2022-2023
7,Quarter-finals,0,2023-04-19,21:00,Inter,Benfica,Stadio Giuseppe Meazza,1.442398,0.778801,Inter,2022-2023


In [38]:
#for input data: 2022-09
df_test

,round,matchweek_no,date,time,home_team,away_team,stadium,home_team_score,away_team_score,winner,season
0,Quarter-finals,0,2023-04-11,20:00,Benfica,Inter,Estádio da Luz (1954),2.049966,0.768270,Benfica,2022-2023
1,Quarter-finals,0,2023-04-11,20:00,Manchester City,Bayern Munich,Etihad Stadium,1.081463,1.580282,Bayern Munich,2022-2023
2,Quarter-finals,0,2023-04-12,21:00,Real Madrid,Chelsea,Estadio Santiago Bernabéu,1.472740,1.291104,Real Madrid,2022-2023
3,Quarter-finals,0,2023-04-12,21:00,Milan,Napoli,Stadio Giuseppe Meazza,0.599859,2.238840,Napoli,2022-2023
4,Quarter-finals,0,2023-04-18,20:00,Chelsea,Real Madrid,Stamford Bridge,1.291104,1.472740,Real Madrid,2022-2023
5,Quarter-finals,0,2023-04-18,21:00,Napoli,Milan,Stadio Diego Armando Maradona,2.238840,0.599859,Napoli,2022-2023
6,Quarter-finals,0,2023-04-19,21:00,Bayern Munich,Manchester City,Allianz Arena,1.580282,1.081463,Bayern Munich,2022-2023
7,Quarter-finals,0,2023-04-19,21:00,Inter,Benfica,Stadio Giuseppe Meazza,0.768270,2.049966,Benfica,2022-2023
